In [6]:
from IPython.display import clear_output
import readline
import numpy as np
from scipy import misc
import scipy
import matplotlib.pyplot as plt
from copy import deepcopy

%matplotlib inline

####Lecture 8: Exceptions, Debugging, and Modular Design

Here's what's planned for today:

Addressing problems before they happen:
1. Asserts!
2. Exceptions! An object we've been running into a lot! 
3. Raising an exception (complaining about a problem)
4. Catching an exception (dealing with a problem)

Dealing with unexpected problems:
1. Finding unexpected problems

Preventing problems:
1. Documentation
2. Modular design

#### Learning Objectives

1. Learn how to think about code structure
2. Assert and test correctness of small pieces of code
3. Assemble hierarchies of functionality to do awesome stuff!

Remember that it pays to think before you type, if you don't you'll just have more work to do later when something breaks!

#### A quick review from last time

##### On Tuesday We Discussed OOP, Classes

OOP allows us to organize our code by thinking about data in terms of actors and actions

Classes *declare kinds* of data, and instances are particular representations of that data:

This is the difference between the class 'Float' and the number 3.14159

In [4]:
# An example application: trigger modeling at the ATLAS detector

# Basic idea, too much data!
# Reason to use OOP: Trigger 'thinks' about what to reject, 
# keeps statistics on what fraction of events it accepts

# particle format
test_particles = [(5.0, 0.0),    # photon?
                  (174.2, 80.8), # w boson?
                  (220.4, 81.8)] # w boson?

In [ ]:
class MassTrigger:
    def __init__(self, cut):
        self._cut = cut
    
    def accepts(self, event):
        masses = np.array(event)[:,-1]
        
        if masses[masses > self._cut].shape()[0] >= 2:
            return True
        return False

# W boson has a mass of 80.4 GeV
two_ws_trigger = MassTrigger(80.4)
print(two_ws_trigger.accepts(test_particles))
print(two_ws_trigger.acceptance_rate())

#### A potential point of potential confusion: Class variables and instance variables

If I changed the total and accepted events... and recorded their ID

In [7]:
class BadMassTrigger:
    """
    A simple trigger that accepts 
    an event if it contains two particles passing a mass cut"""

    accepted_event_numbers = []
    
    def __init__(self, cut):
        self.accepted_evts = 0
        self.total_evts = 0
        self.cut = cut
    
    def accepts(self, event, event_number):
        self.total_evts += 1
        np_event = np.array(event)
        if np_event[:,-1][np_event[:,-1] > self.cut].shape[0] >= 2:
            self.accepted_evts += 1
            self.accepted_event_numbers.append(event_number)
            return True
        return False
    
    def acceptance_rate(self):
        return float(self.accepted_evts) / self.total_evts

In [8]:
trig_w = BadMassTrigger(80.4)
trig_lowmass = BadMassTrigger(10.0)

print(trig_w.accepts(test_particles, 0))
print(trig_lowmass.accepts(test_particles, 0))

print(trig_w.accepted_event_numbers)

True
True
[0, 0]


##### Classes are very useful!

OOP is a fairly natural way of organizing how you think about your code, but don't go overboard. Sometimes achieving the kind of data/method locality required by OOP forces you to program unnaturally.

It is better to do something *simply* and *naturally* than to try to shove it into a particular paradigm.



In [11]:
# Okay... new material
# Who remembers seeing exceptions?
1 / 0

ZeroDivisionError: division by zero

##### Exceptions get raised in lots of circumstances

Type errors, index errors, bad file read... plenty of others!

How do you think they are organized?

In [2]:
def p():
    pass
p(5)

TypeError: p() takes 0 positional arguments but 1 was given

In [15]:
def only_adds_odds(a, b):
    if a % 2 == 0 or b % 2 == 0:
        raise TypeError()
    return a + b
    
lvs = [1, 7, 13, 28, 5]
rvs = [9, 0, -5, 28, 59]

for a, b in zip(lvs, rvs):
    try:
        print(only_adds_odds(a, b))
    except:
        print("One of those was even.")

10
One of those was even.
8
One of those was even.
64


#### Okay but what *are* Exceptions?

Exceptions are just instances of some particular class.

They hold information related to the programmatic context in which something went wrong, as well as what in particular happened.

Because different *kinds* of bad things can happen, there are different classes of exceptions.

Our "I can't even." ValueError occurred because our function hated even numbers, but ValueErrors occur whenever a piece of code encounters an unexpected value.

This is the difference between a class (TypeError) and an instance of that class (TypeError("Some complaint")

#### Let's Tear an Exception Apart in IPython

#### Some advice

We saw that we can catch the most general exception! "except Exception:"

Don't do this! Only make handlers for things you're actually handling!

To do otherwise defeats the purpose of the exceptions, to let you know something bad is happening!

#### What you should do...

Use assertions!

In [17]:
def make_sure_its_zero(a):
    """Ensures it's argument is zero using the 
    Python language feature, 'assert' """
    assert a == 0
    

def alternate(a):
    """Makes sure it's argument is zero 
    by directly raising an AssertionError"""
    if a != 0:
        raise AssertionError()

make_sure_its_zero(1)

AssertionError: 

#### Why Assertions?

This is a good question!
In order to really answer this adequately I want to take a step back and talk about modular program design.

##### Modularization and Abstraction Management
OOP and other programming paradigms give us ways of modularizing code and managing abstractions.

You should think of OOP as a way of designing, and as a tool to be situated among others Python offers:

1. Module level design
2. Classes/OOP + Functional (we'll talk about functional next week)
3. Functions
4. Python builtins

If you work in IPython Notebook, there is also the 'cell abstraction'


##### Hierarchies of Scope
Each level of this hierarchy has a different scope, but in each case it's narrower than that of the 'full' program

HOWEVER

Scope limitations don't exist to the computer, design abstractions only exist to benefit the *readers* and *writers* of the program

You have to *assert* some set of invariants to declare these limitations of scope, and thereby restrict behavior.

It's a sad reality that the space of valid computations achievable with the program you write is larger than the set you were probably trying to address! Possible bugs!

##### 'Asserting' Hierarchies
Type systems, as in Java and other statically typed languages, are one way of declaring these invariants.

Asserts are also a very common way of doing this.

Here are some reasonble uses of things to be checked with assertions:

1. Parameter types, classes, or values
2. Data structure invariants
3. Presence of unintended manipulations to mutable data
4. Reasonability of return results

Do you notice anything about these in particular? (except maybe 2?)

## Avoiding Errors in the First Place

We don't have to deal with exceptions as much if we prevent errors from occurring in the first place!

Python makes this easier than most languages, because it has a REPL:

Encourages the following workflow:

1. Pick a small piece of the problem
2. Write/fix code to solve it
3. Test it in the REPL! (Go back to 2 if necessary.)

In particular... DON'T test the full system

Given how easy it is to test code by hand in Python, there's really no excuse to write a lot of code and cross your fingers that it works

If you find that you can't get to a problem quickly by playing with it in a REPL, there are lots of tools available to you find and fix issues. One beautiful one is pdb.

#### Review:

Keep in mind that classes, modules (which are just Python files), and functions allow you to structure the code that you write.

Use asserts/exceptions, hand debugging in a REPL, and pdb to figure out what went wrong

Document your code to make your (and your readers') lives easier!

In [ ]:
# Classes
class MyClass:
    def __init__(self, ...):
        pass
    ...

# Exceptions
raise [some exception object] # e.g. raise ZeroDivisionError()

# Undertaking a dangerous operation:
try:
    # some stuff here
    pass
except [ExceptionType] [optional: as e]:
    # handle e
    pass

# Asserting
assert [some boolean condition]

# Documenting
def my_fn(a, b, c):
    """
    my_fn uses the numerical value provided through a and blah blah blah...
    returns blah blah blah
    """
    pass

# IPython
# Tab completion! Question mark! Exclamation point!

# OPTIONAL
# pdb
# s, c, p, bt, %debug (IPython magic), pdb.set_trace()


#### Next Week

We have two principal goals next week

1. Advanced OOP
2. Functional programming 

You will see how these tools let you write less code, and instead leverage what's already built into Python!
